In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense,LSTM
from sklearn import metrics
import math
from pandas import DataFrame
from pandas import concat

In [ ]:
df = pd.read_csv('SARS-CoV-2 Dataset Updated (1).csv')
countries = df[~df['continent'].isnull()]['location'].unique()
continents =  df[df['continent'].isnull()]['location'].unique()
continents

In [ ]:
df = df.fillna(0)
sub_df = df[['location', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_cases', 'new_deaths']]


In [ ]:
def fill_null_rows(idx, dataframe):
    total_vaccinations = dataframe.at[idx, 'total_vaccinations']
    people_vaccinated = dataframe.at[idx, 'people_vaccinated']
    people_fully_vaccinated = dataframe.at[idx, 'people_fully_vaccinated']
    new_cases = dataframe.at[idx, 'new_cases']
    new_deaths = dataframe.at[idx, 'new_deaths']
    
    if dataframe.at[idx - 1, 'location'] == dataframe.at[idx, 'location']:
        if total_vaccinations == 0:
            dataframe.at[idx, 'total_vaccinations'] = dataframe.at[idx - 1, 'total_vaccinations']
        if people_vaccinated == 0:
            dataframe.at[idx, 'people_vaccinated'] = dataframe.at[idx - 1, 'people_vaccinated']
        if people_fully_vaccinated == 0:
            dataframe.at[idx, 'people_fully_vaccinated'] = dataframe.at[idx - 1, 'people_fully_vaccinated']
        if new_cases == 0:
            dataframe.at[idx, 'new_cases'] = dataframe.at[idx - 1, 'new_cases']
        if new_deaths == 0:
            dataframe.at[idx, 'new_deaths'] = dataframe.at[idx - 1, 'new_deaths']

In [ ]:
for index, row in sub_df.iterrows():
    if index == 0:
        continue
    fill_null_rows(index, sub_df)

In [ ]:
df[['location', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'new_cases', 'new_deaths']] = sub_df

In [ ]:
df.drop(df[df.location == 'International'].index, inplace=True)
df.drop(df[df.location == 'Northern Cyprusn'].index, inplace=True)

# LSTM 2.1.

In [ ]:
continent = "Africa"
country = "United States"
target ="new_cases"
df_country = df[df["location"] == country]
df_country = df_country[['date', target]]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')
x = len(df_country)-30
train=df_country.iloc[:x]
test = df_country.iloc[x:]

scaler = MinMaxScaler()
scaler.fit(train) 
train_scaled = scaler.transform(train)
test_scaled = scaler.transform(test)

seq_size = 7
n_features = 1
train_generator = TimeseriesGenerator(train_scaled, train_scaled, length = seq_size, batch_size=1)
test_generator = TimeseriesGenerator(test_scaled, test_scaled, length=seq_size, batch_size=1)
model = Sequential()
model.add(LSTM(150, activation='relu', return_sequences=True, input_shape=(seq_size, n_features)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(64))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_generator, 
                    validation_data=test_generator, 
                    epochs=50, steps_per_epoch=10)
prediction = []
current_batch = train_scaled[-seq_size:]
current_batch = current_batch.reshape(1, seq_size, n_features)
days = 30
for i in range(days):
    current_pred = model.predict(current_batch)[0]
    prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
rescaled_prediction = scaler.inverse_transform(prediction)
time_series_array = df_country.index[:len(df_country)-(30-days)]

    
predict = pd.DataFrame(rescaled_prediction,columns=[target], index=test.index[:days])

df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)

df_forecast.loc[:,"predicted"] = predict[target]
df_forecast.loc[:,"actual"] = df_country[target]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{days}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

In [ ]:
MAE_lstm = metrics.mean_absolute_error(test, predict)
MSE_lstm = metrics.mean_squared_error(test, predict)
RMSE_lstm = np.sqrt(metrics.mean_squared_error(test, predict))
R2_lstm = metrics.r2_score(test, predict)
#test['new_cases']=test.reshape(len(test),1)
RAE_lstm = np.sum(np.abs(np.subtract(test,predict))) / np.sum(np.abs(np.subtract(test, np.mean(test))))
RAE_lstm = RAE_lstm[target].astype('float')
RRSE_lstm = np.sqrt(np.sqrt(MSE_lstm/(len(test)-2)))

print("MAE : ",MAE_lstm)
print("MSE : ",MSE_lstm)
print("RMSE : ",RMSE_lstm)
print("R2 : ",R2_lstm)
print("RAE : ",RAE_lstm)
print("RRSE : ",RRSE_lstm)

# 2.2.

In [ ]:
continent = "Africa"
country = "United States"
target ="people_fully_vaccinated"
df_country = df[df["location"] == continent]
df_country = df_country[['date', target]]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')
x = len(df_country)-30

scaler = MinMaxScaler()
scaler.fit(df_country) 
train_scaled = scaler.transform(df_country)

seq_size = 7
n_features = 1
train_generator = TimeseriesGenerator(train_scaled, train_scaled, length = seq_size, batch_size=1)
model = Sequential()
model.add(LSTM(150, activation='relu', return_sequences=True, input_shape=(seq_size, n_features)))
model.add(LSTM(64, activation='relu'))
model.add(Dense(64))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(train_generator,epochs=50, steps_per_epoch=10)

prediction = []
current_batch = train_scaled[-seq_size:]
current_batch = current_batch.reshape(1, seq_size, n_features)
days = 30
for i in range(days):
    current_pred = model.predict(current_batch)[0]
    prediction.append(current_pred)
    current_batch = np.append(current_batch[:,1:,:],[[current_pred]],axis=1)
rescaled_prediction = scaler.inverse_transform(prediction)
time_series_array = df_country.index[:len(df_country)-(30-days)]
for k in range(0, days):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))    

df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
predict = pd.DataFrame(rescaled_prediction,columns=[target], index=df_forecast.index[len(df_forecast)-days:])
df_forecast.loc[:,"predicted"] = predict[target]
df_forecast.loc[:,"actual"] = df_country[target]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
fig.savefig(f'images/{days}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

# Random Forest.

In [ ]:
country = "United States"
continent = "Asia"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=1
n_out=2
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)

s = len(agg)-30
train=agg.iloc[:s]
test = agg.iloc[s:]

y = train["var1(t+"+str(n_out-1)+")"].values
x = train.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
x_test =test.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
y_test =test["var1(t+"+str(n_out-1)+")"].values

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 30)
regressor.fit(x, y) 
predict = regressor.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]
predict = pd.DataFrame(predict[:days],columns=["var1(t+"+str(n_out-1)+")"], index=test.index[:days])
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{future}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

In [ ]:
MAE_RF = metrics.mean_absolute_error(y_test, predict)
MSE_RF = metrics.mean_squared_error(y_test, predict)
RMSE_RF = np.sqrt(metrics.mean_squared_error(y_test, predict))
R2_RF = metrics.r2_score(y_test, predict)
y_test=y_test.reshape(len(y_test),1)
RAE_RF = np.sum(np.abs(np.subtract(y_test,predict))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RAE_RF = RAE_RF["var1(t+"+str(n_out-1)+")"].astype('float')
RRSE_RF = np.sqrt(np.sqrt(MSE_RF/(len(y_test)-2)))

print("MAE : ",MAE_RF)
print("MSE : ",MSE_RF)
print("RMSE : ",RMSE_RF)
print("R2 : ",R2_RF)
print("RAE : ",RAE_RF)
print("RRSE : ",RRSE_RF)
#fig = plt.figure()
#ax = fig.add_axes([0,0,1,1])
#x = ['MAE','MSE','RMSE', 'R2','RAE','RRSE']
#y = [MAE,MSE,RMSE,R2,RAE,RRSE]
#ax.bar(x,y)

In [ ]:
country = "United States"
continent = "Africa"
target ="new_cases"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=1
n_out=2
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)
    
y = agg["var1(t+"+str(n_out-1)+")"].values
x = agg.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
#x_test =test.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
#y_test =test[target].values

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.044)

from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 100, random_state = 30)
regressor.fit(x, y) 
predict = regressor.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]
for k in range(0, days):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))
    
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
predict = pd.DataFrame(predict,columns=["var1(t+"+str(n_out-1)+")"], index=df_forecast.index[len(df_forecast)-days:])
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{days}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")


# Desicion Tree.

In [ ]:
country = "United States"
continent = "Africa"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=6
n_out=6
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)

s = len(agg)-30
train=agg.iloc[:s]
test = agg.iloc[s:]

y = train["var1(t+"+str(n_out-1)+")"].values
x = train.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
x_test =test.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
y_test =test["var1(t+"+str(n_out-1)+")"].values


from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 30)
regressor.fit(x, y) 
predict = regressor.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]

    
predict = pd.DataFrame(predict[:days],columns=["var1(t+"+str(n_out-1)+")"], index=test.index[:days])
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{future}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

In [ ]:
MAE_DT = metrics.mean_absolute_error(y_test, predict)
MSE_DT = metrics.mean_squared_error(y_test, predict)
RMSE_DT = np.sqrt(metrics.mean_squared_error(y_test, predict))
R2_DT = metrics.r2_score(y_test, predict)
y_test=y_test.reshape(len(y_test),1)
RAE_DT = np.sum(np.abs(np.subtract(y_test,predict))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RAE_DT = RAE_DT["var1(t+"+str(n_out-1)+")"].astype('float')
RRSE_DT = np.sqrt(np.sqrt(MSE_DT/(len(y_test)-2)))

print("MAE : ",MAE_DT)
print("MSE : ",MSE_DT)
print("RMSE : ",RMSE_DT)
print("R2 : ",R2_DT)
print("RAE : ",RAE_DT)
print("RRSE : ",RRSE_DT)
#fig = plt.figure()
#ax = fig.add_axes([0,0,1,1])
#x = ['MAE','MSE','RMSE', 'R2','RAE','RRSE']
#y = [MAE,MSE,RMSE,R2,RAE,RRSE]
#ax.bar(x,y)

In [ ]:
country = "Afghanistan"
continent = "Africa"
target ="total_deaths"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=1
n_out=2
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)
    
y = agg["var1(t+"+str(n_out-1)+")"].values
x = agg.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.046)

from sklearn.tree import DecisionTreeRegressor
regressor = DecisionTreeRegressor(random_state = 30)
regressor.fit(x, y) 
predict = regressor.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]
for k in range(0, days):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))
    
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
predict = pd.DataFrame(predict,columns=["var1(t+"+str(n_out-1)+")"], index=df_forecast.index[len(df_forecast)-days:])
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{days}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

# NAIVE BAYES.

In [ ]:
from sklearn.naive_bayes import GaussianNB

country = "United States"
continent = "Africa"
target ="new_cases"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=5
n_out=6
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)

s = len(agg)-30
train=agg.iloc[:s]
test = agg.iloc[s:]

y = train["var1(t+"+str(n_out-1)+")"].values
x = train.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
x_test =test.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)
y_test =test["var1(t+"+str(n_out-1)+")"].values

gnb = GaussianNB()
gnb.fit(x,y)
predict = gnb.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]
predict = pd.DataFrame(predict[:days],columns=["var1(t+"+str(n_out-1)+")"], index=test.index[:days])
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{future}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

In [ ]:
MAE_NB = metrics.mean_absolute_error(y_test, predict)
MSE_NB = metrics.mean_squared_error(y_test, predict)
RMSE_NB = np.sqrt(metrics.mean_squared_error(y_test, predict))
R2_NB = metrics.r2_score(y_test, predict)
y_test=y_test.reshape(len(y_test),1)
RAE_NB = np.sum(np.abs(np.subtract(y_test,predict))) / np.sum(np.abs(np.subtract(y_test, np.mean(y_test))))
RAE_NB = RAE_NB["var1(t+"+str(n_out-1)+")"].astype('float')
RRSE_NB = np.sqrt(np.sqrt(MSE_NB/(len(y_test)-2)))

print("MAE : ",MAE_NB)
print("MSE : ",MSE_NB)
print("RMSE : ",RMSE_NB)
print("R2 : ",R2_NB)
print("RAE : ",RAE_NB)
print("RRSE : ",RRSE_NB)
#fig = plt.figure()
#ax = fig.add_axes([0,0,1,1])
#x = ['MAE','MSE','RMSE', 'R2','RAE','RRSE']
#y = [MAE,MSE,RMSE,R2,RAE,RRSE]
#ax.bar(x,y)

In [ ]:
country = "Afghanistan"
continent = "Africa"
target ="total_deaths"
df_country = df[df["location"] == country]
df_country = df_country[['date','new_cases']]
df_country.set_index('date', inplace=True)
df_country.index = pd.to_datetime(df_country.index,format='%m/%d/20%y')

n_in=1
n_out=2
dropnan=True
n_vars = 1 if type(df_country) is list else df_country.shape[1]
cols, names = list(), list()
for i in range(n_in, 0, -1):
    cols.append(df_country.shift(i))
    names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
for i in range(0, n_out):
    cols.append(df_country.shift(-i))
    if i == 0:
        names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
    else:
        names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
agg = concat(cols, axis=1)
agg.columns = names
if dropnan:
    agg.dropna(inplace=True)
    
y = agg["var1(t+"+str(n_out-1)+")"].values
x = agg.drop(labels=["var1(t+"+str(n_out-1)+")"],axis=1)

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.046)

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(x,y)
predict = gnb.predict(x_test)

days = 30
time_series_array = agg.index[:len(agg)-(30-days)]
for k in range(0, days):
    time_series_array = time_series_array.append(time_series_array[-1:] + pd.DateOffset(1))
    
df_forecast = pd.DataFrame(columns=["actual","predicted"], index=time_series_array)
predict = pd.DataFrame(predict,columns=["var1(t+"+str(n_out-1)+")"], index=df_forecast.index[len(df_forecast)-days:])
df_forecast.loc[:,"predicted"] = predict["var1(t+"+str(n_out-1)+")"]
df_forecast.loc[:,"actual"] = agg["var1(t+"+str(n_out-1)+")"]
fig = df_forecast.plot(kind='line',  
figsize=(50, 16), fontsize=16).get_figure()
#fig.savefig(f'images/{days}-{country}.png')
#df_forecast.plot(title="Predictions for next 7 days")

In [ ]:
#MAE
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['MAE_lstm','MAE_RF','MAE_DT', 'MAE_NB']
y = [MAE_lstm,MAE_RF,MAE_DT,MAE_NB]
ax.bar(x,y)

#MSE
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['MSE_lstm','MSE_RF','MSE_DT', 'MSE_NB']
y = [MSE_lstm,MSE_RF,MSE_DT,MSE_NB]
ax.bar(x,y)

#RMSE
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['RMSE_lstm','RMSE_RF','RMSE_DT', 'RMSE_NB']
y = [RMSE_lstm,RMSE_RF,RMSE_DT,RMSE_NB]
ax.bar(x,y)

#R2
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['R2_lstm','R2_RF','R2_DT', 'R2_NB']
y = [R2_lstm,R2_RF,R2_DT,R2_NB]
ax.bar(x,y)

#RAE
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['RAE_lstm','RAE_RF','RAE_DT','RAE_NB']
y = [RAE_lstm,RAE_RF,RAE_DT,RAE_NB]
ax.bar(x,y)

#RRSE
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
x = ['RRSE_lstm','RRSE_RF','RRSE_DT', 'RRSE_NB']
y = [RRSE_lstm,RRSE_RF,RRSE_DT,RRSE_NB]
ax.bar(x,y)